In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from unidecode import unidecode
import shutil
from pathlib import Path

link = 'https://www.imdb.com/'

res = requests.get(f'{link}chart/top/?ref_=nv_mv_250')
soup = BeautifulSoup(res.text, 'html5lib')

In [ ]:
tabela = soup.find('tbody', {'class': 'lister-list'})
filmes = tabela.find_all('tr')
Path('./posters').mkdir(exist_ok=True)

filmes_dic = {}

for filme in filmes:
    
    filme_site = requests.get(f"{link}{filme.find('a')['href']}")
    filme_html = BeautifulSoup(filme_site.text, 'html5lib')
    titulo = filme_html.find('h1', attrs={'data-testid': 'hero-title-block__title'}).text
    valores = {}
    valores['Ano de lancamento'] = filme_html.find('ul', attrs={'data-testid': 'hero-title-block__metadata'}).find('a').text
    valores['URL do poster'] = filme_html.find('div', attrs={'data-testid': 'hero-media__poster'}).find('a')['href']
    valores['Imagem do poster'] = filme_html.find('div', attrs={'data-testid': 'hero-media__poster'}).find('img')['src']
    
    res_img = requests.get(valores['Imagem do poster'], stream=True)
    img_path = f'./posters/{unidecode(titulo)}.png'
    with open(img_path, 'wb') as f:
            shutil.copyfileobj(res_img.raw, f)
    valores['Caminho da imagem'] = img_path

    valores['Nota'] = filme_html.find('div', attrs={'data-testid': 'hero-rating-bar__aggregate-rating__score'}).find('span').text

    generos = []
    for genero in filme_html.find('div', attrs={'data-testid': 'genres'}).find_all('a'):
        generos.append(genero.text)

    valores['Generos'] = generos

    diretores = []
    for diretor in filme_html.find('section', attrs={'data-testid': 'title-cast'}).find('div', {'class': 'ipc-metadata-list-item__content-container'}).find_all('li'):
        diretores.append(diretor.text)

    valores['Diretores'] = diretores

    filmes_dic[unidecode(titulo)] = valores

In [ ]:
with open("sample.json", "w") as outfile:
    json.dump(filmes_dic, outfile)